#Все импорты

In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import pandas as pd
import numpy as np

#Подключение к г. диску, формирование датафреймов

In [2]:
data_path = '/content/drive/MyDrive/ML/COOKIE/info/data/'

df = pd.read_csv(data_path + 'act_dataset/телеметрия/data2024-06-01.csv')
vib_time = pd.read_csv(data_path + 'act_dataset/данные_вибрации/2024_06_01/times.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29731 entries, 0 to 29730
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   t                             29731 non-null  object 
 1   v_gas                         29731 non-null  float64
 2   v_pres_gauge                  29731 non-null  float64
 3   v_liq_flowrate                29731 non-null  float64
 4   p_atma                        29731 non-null  float64
 5   rl                            29731 non-null  float64
 6   rg                            29731 non-null  float64
 7   rm                            29731 non-null  float64
 8   mul                           29731 non-null  float64
 9   mum                           29731 non-null  float64
 10  mug                           29731 non-null  float64
 11  gf                            29731 non-null  float64
 12  stlg                          29731 non-null  float64
 13  s

In [4]:
df.t

,t
0,2024-06-01 11:33:26.350
1,2024-06-01 11:36:55.168
2,2024-06-01 11:39:46.884
3,2024-06-01 11:46:06.207
4,2024-06-01 12:30:47.376
...,...
29726,2024-06-01 18:06:57.844
29727,2024-06-01 18:07:01.776
29728,2024-06-01 18:07:02.494
29729,2024-06-01 18:07:03.124


#Вспомогательные функции

In [5]:
# corr matrix
def print_corr_matrix(df, title):
  num_df = df.select_dtypes(include=['number'])
  df_corr = round(num_df.corr(), 3)

  plt.figure(figsize=(20, 10))
  sns.heatmap(df_corr, annot=True, cmap='coolwarm')

  plt.title(title)
  plt.show()

#IQR outliers
def get_outliers_idx(df, feature):
  IQR = df[feature].quantile(0.75) - df[feature].quantile(0.25)
  low = df[feature].quantile(0.25) - (IQR * 1.5)
  up = df[feature].quantile(0.75) + (IQR * 1.5)
  borders = (low, up)
  outliers = pd.concat([df[feature] < low, df[feature] > up], axis=1)
  outliers_index = outliers.any(axis=1)
  return outliers_index

def print_box_plots(df):
  for column in df.columns:
    fig = px.box(df, y=column)
    fig.update_layout(title=column)
    fig.show()

#3-sigma
def three_sigma_otliners(df, feature):
  mean = df[feature].mean()
  std_dev = df[feature].std()
  lower_bound = mean - 3 * std_dev
  upper_bound = mean + 3 * std_dev

  outliers = pd.concat([df[feature] < lower_bound, df[feature] > upper_bound], axis=1)
  outliers_index = outliers.any(axis=1)
  return outliers_index

def drop_otliners(df, method):
  new_df = df

  for column in new_df.columns:
    if column == 't':
      continue

    if method == 'sigma':
      new_df = new_df[~three_sigma_otliners(new_df, column)]
    elif method == 'IQR':
      new_df = new_df[~get_outliers_idx(new_df, column)]
    else:
      print('Хуйню в параметры передал')

  new_df = new_df.reset_index(drop=True)
  return new_df

# df1 06-01

In [6]:
vib_time['start_time'] = pd.to_datetime(vib_time['start_time'])
vib_time['end_time'] = pd.to_datetime(vib_time['end_time'])

df.drop('rs', axis=1, inplace=True)
df['t'] = pd.to_datetime(df['t'])
df['t'] = df['t'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
df['t'] = pd.to_datetime(df['t'])

In [7]:
def check_vibs(row, vib_time):
    for _, interval in vib_time.iterrows():
        if interval['start_time'] <= row['t'] <= interval['end_time']:
            return 1
    return 0

# Применяем функцию для создания новой колонки vibs
df['vibs'] = df.apply(lambda row: check_vibs(row, vib_time), axis=1)

In [8]:
vibs_df = df[df['vibs'] == 1]

In [9]:
vibs_df

,t,v_gas,v_pres_gauge,v_liq_flowrate,p_atma,rl,rg,rm,mul,mum,...,gas_flowrate_m3day,qm_m3day,superficial_liquid_velocity,superficial_gas_velocity,superficial_mixture_velocity,Re_liq_value,Re_mix_value,Fr_value,We_value,vibs
10352,2024-06-01 14:33:20,0.000000,1.203629,249.451248,2.203629,998.458943,1.843695,998.458943,0.973919,0.973919,...,0.000000,249.836260,1.472692,0.000000,1.472692,75489.932520,75489.932520,4.421653,1534.445918,1
10353,2024-06-01 14:33:20,79.514633,1.205380,0.000000,2.205380,998.458956,1.845169,1.845169,0.973920,0.010509,...,36.348219,36.348219,0.000000,0.214259,0.214259,0.000000,1880.924058,0.000000,0.000000,1
10354,2024-06-01 14:33:21,79.622635,1.213779,0.000000,2.213779,998.459018,1.852243,1.852243,0.973925,0.010509,...,36.258576,36.258576,0.000000,0.213731,0.213731,0.000000,1883.467200,0.000000,0.000000,1
10355,2024-06-01 14:33:21,0.000000,1.214829,249.144089,2.214829,998.459026,1.853128,998.459026,0.973926,0.973926,...,0.000000,249.528606,1.470878,0.000000,1.470878,75396.476530,75396.476530,4.410770,1530.676407,1
10356,2024-06-01 14:33:21,79.796837,1.218328,0.000000,2.218328,998.459052,1.856075,1.856075,0.973928,0.010509,...,36.262880,36.262880,0.000000,0.213756,0.213756,0.000000,1887.581605,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26354,2024-06-01 17:12:29,87.356567,3.413659,132.984375,4.413659,998.475620,3.717669,869.620147,0.975203,0.850244,...,19.819698,153.007101,0.785090,0.116830,0.901920,40191.306454,46123.674956,1.256609,436.489883,1
26355,2024-06-01 17:12:30,87.284927,3.415759,0.000000,4.415759,998.475636,3.719462,3.719462,0.975204,0.010529,...,19.793899,19.793899,0.000000,0.116678,0.116678,0.000000,2060.824505,0.000000,0.000000,1
26356,2024-06-01 17:12:30,0.000000,3.410159,130.054535,4.410159,998.475593,3.714682,998.475593,0.975201,0.975201,...,0.000000,130.253094,0.767793,0.000000,0.767793,39305.915350,39305.915350,1.201849,417.468131,1
26357,2024-06-01 17:12:30,87.572052,3.411909,0.000000,4.411909,998.475606,3.716176,3.716176,0.975202,0.010529,...,19.876573,19.876573,0.000000,0.117165,0.117165,0.000000,2067.611275,0.000000,0.000000,1


In [13]:
df = pd.read_csv(data_path + 'act_dataset/телеметрия/data2024-06-01.csv')

df.drop('rs', axis=1, inplace=True)
df['t'] = pd.to_datetime(df['t'])
df['t'] = df['t'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
df['t'] = pd.to_datetime(df['t'])

sigma_no_outliners_df = drop_otliners(df, 'sigma')

sigma_no_outliners_df['vibs'] = sigma_no_outliners_df.apply(lambda row: check_vibs(row, vib_time), axis=1)

sigma_no_outliners_df['vibs'].sum()

2508

In [14]:
sigma_no_outliners_df = sigma_no_outliners_df[sigma_no_outliners_df['vibs'] == 1]

In [16]:
out_df = pd.DataFrame()
out_df['t'] = sigma_no_outliners_df['t']
out_df['gas_flowrate_m3day'] = sigma_no_outliners_df['gas_flowrate_m3day']
out_df['liquid_flowrate_m3day'] = sigma_no_outliners_df['liquid_flowrate_m3day']


out_df = out_df.groupby(out_df['t']).mean()
out_df = out_df.reset_index()
out_df.to_csv('vibrations_ds.csv')